# Лабораторная работа 2. Задача о рюкзаке. 

### Мельников Евгений 18ПМИ-1, Екатерина Малышева 18ПМИ-2

In [ ]:
import time
from google.colab import drive
import os
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
from math import gcd, floor
from random import randint
import time
from itertools import *
from scipy.optimize import linprog
import copy
 
drive.mount('/drive')
os.listdir('/drive/MyDrive/knapsnack_benchmarks')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


['p01_c.txt',
 'p01_w.txt',
 'p01_p.txt',
 'p01_s.txt',
 'p02_c.txt',
 'p02_w.txt',
 'p02_p.txt',
 'p02_s.txt',
 'p03_c.txt',
 'p03_w.txt',
 'p03_p.txt',
 'p03_s.txt',
 'p04_c.txt',
 'p04_w.txt',
 'p04_p.txt',
 'p04_s.txt',
 'p05_c.txt',
 'p05_w.txt',
 'p05_p.txt',
 'p05_s.txt',
 'p06_c.txt',
 'p06_w.txt',
 'p06_p.txt',
 'p06_s.txt',
 'p07_c.txt',
 'p07_w.txt',
 'p07_p.txt',
 'p07_s.txt']

## Dynamic programming

In [1]:
def dynamic_programming_algo(capacity, weight, profit):
  total_operations = 0

  previous_row = [] 
  current_row = [] # only current and previous rows are used so that we don't store unnecessary table data
  current_row_items = []
  previous_row_items = []
  total_items = len(weight)

  for i in range(capacity + 1):
    previous_row.append(0)
    current_row.append(0)
    current_row_items.append([])
    previous_row_items.append([])

  start = time.time()
  for items_amount in range(total_items + 1):
    for current_capacity in range(capacity + 1):

      total_operations += 1

      #print(current_capacity)
      if items_amount == 0 or current_capacity == 0:
        current_row[current_capacity] = 0
      elif current_capacity >= weight[items_amount - 1]:  # w >= w_i 
        #print("taking item number", items_amount - 1)    
        total_operations += 1 #?
        if profit[items_amount - 1] + previous_row[current_capacity - weight[items_amount - 1]] > previous_row[current_capacity]:
          current_row[current_capacity] = profit[items_amount - 1] + previous_row[current_capacity - weight[items_amount - 1]]
          current_row_items[current_capacity] = previous_row_items[current_capacity - weight[items_amount - 1]].copy()
          current_row_items[current_capacity].append(items_amount - 1)
        else:                                                    
          current_row[current_capacity] = previous_row[current_capacity]
          current_row_items[current_capacity] = previous_row_items[current_capacity]
      elif current_capacity < weight[items_amount - 1]:
        current_row[current_capacity] = previous_row[current_capacity] # we don't take the item
        current_row_items[current_capacity] = previous_row_items[current_capacity]
    previous_row = current_row.copy()
    previous_row_items = current_row_items.copy()
    #print(current_row)  
    #print(current_row_items)
  return current_row_items[capacity], current_row[capacity], time.time() - start, total_operations # returns the items and their total weight
  

## PTAS

In [ ]:
# copied this one from a website
def all_subsets(array, subset_size): # wierd af
  array_size = len(array)
  data = list(range(subset_size))

  subsets = []
  def all_subsets_rec(index, data, i):

    #print(array)
    if index == subset_size:
      subsets.append(data.copy())
      return
 
    # When no more elements
    # are there to put in data[]
    if i >= array_size:
      return
 
    # current is included,
    # put next at next
    # location
    data[index] = array[i]
    
    all_subsets_rec(index + 1, data, i + 1) 
    # current is excluded,
    # replace it with
    # next (Note that i+1
    # is passed, but index
    # is not changed)

    all_subsets_rec(index, data, i + 1)  
  all_subsets_rec(0, data, 0)
  return subsets


def greedy_search(capacity, weights, profits, subset): # subset of items (M)
  resulting_profit = 0 # z^g
  remaining_capacity = capacity # c hat
  for item in subset:
    remaining_capacity -= weights[item]
    #resulting_profits += profits[item] # ?

  X = [] # ! gotta be renamed

  for item in range(len(weights)):
    if item not in subset and weights[item] <= remaining_capacity: # if item is not in the subset in fits in the leftover space
      resulting_profit += profits[item]
      remaining_capacity -= weights[item]
      X.append(item)

  return resulting_profit, X


def ptas(capacity, weights, profits, subset_size):
  total_operations = 0

  items = list(range(len(weights)))
  subsets = all_subsets(items, subset_size)
  best_profit = 0
  best_X = []

  start = time.time()

  for subset in subsets:
    subset_weight = 0
    subset_profit = 0
    #best_X = []
    for item in subset:
      subset_weight += weights[item]
      subset_profit += profits[item]

    total_operations += 1
    if subset_weight <= capacity:
      resulting_profit, X = greedy_search(capacity, weights, profits, subset)
      total_operations += 1      
      if resulting_profit + subset_profit > best_profit:
        best_profit = resulting_profit + subset_profit
        #print(subset, X)
        best_X = list(set(X) | set(subset))
  #return best_X, best_profit      
  return best_X, best_profit, time.time() - start, total_operations 

# 2-approx


In [ ]:

def approx_2(capacity, weights, profits):
  quality = []
  num_items = len(weights)
  count_operations=0
  start=time.time()
  for i in range(num_items):
    quality.append(profits[i]/weights[i])
  quality_sorted_arg = np.argsort(quality)
  quality_sorted_arg = np.flip(quality_sorted_arg)
  profit_sorted_arg = np.argsort(profits)
  profit_sorted_arg = np.flip(profit_sorted_arg )

  quality_c, profits_c= 0, 0
  quality_w, profits_w= 0, 0
  backpack_by_quality=[]
  backpack_by_profits=[]

  for i in range(num_items):
    count_operations+=2
    if quality_w + weights[quality_sorted_arg[i]] <= capacity:
      quality_c += profits[quality_sorted_arg[i]]
      quality_w += weights[quality_sorted_arg[i]] 
      backpack_by_quality.append(quality_sorted_arg[i])
    if profits_w + weights[profit_sorted_arg[i]] <= capacity:
      profits_c += profits[profit_sorted_arg[i]]
      profits_w += weights[profit_sorted_arg[i]] 
      backpack_by_profits.append(profit_sorted_arg[i])
  if quality_c >= profits_c:
    return sorted(backpack_by_quality), quality_c, time.time()-start, count_operations
  else:
    return sorted(backpack_by_profits), profits_c, time.time()-start, count_operations



# Branch and bound method

In [ ]:
class node:
    
    def __init__(self, func, weights, profits, bounds, capacity, A_equal, b_equal ):
        if A_equal and b_equal:
            opt = linprog(c = func, A_ub = [weights], b_ub = capacity , A_eq = A_equal, b_eq = b_equal, bounds = bounds, method = 'simplex')
        else:
            opt = linprog(c = func, A_ub = [weights], b_ub = capacity, bounds = bounds, method = 'simplex')
            
        self.fun = opt.fun
        self.items = []
        self.num_ind = -1 #нецелочисленное 
        
        for i in range(len(opt.x)):
            if abs(1 - opt.x[i]) > 0.01:
                self.items.append(opt.x[i])
            else:
                self.items.append(round(opt.x[i]))
            if self.items[i] > 0 and self.items[i] < 1:
                self.num_ind= i #позиция нецелого элемента
                
        self.tot_weight=0
        for i in range(len(self.items)):
            self.tot_weight+= self.items[i]* weights[i]
        
            
        self.A_equal = A_equal
        self.b_equal = b_equal
  
def bbm(capacity, weights, profits):
    start=time.time()
    func=[]
    for p in profits:
        func.append(-p)
        
    bounds = []
    for i in range(len(weights)):
        bounds.append((0, 1))
    
    global best_profit
    
    best_profit = 0
    global best_backpack 
    best_backpack = []
    global num_operations 
    num_operations = 0
    
    A_equal=[]
    b_equal=[]
    
    queue=[]
    first_sol=node(func, weights, profits, bounds, capacity, [], [])
    
    
    queue.append(first_sol)
    num_op=0
    while queue:
        num_op+=1
        
        cur_node = queue.pop(0)
        #print(cur_node.items)
        
        if cur_node.num_ind == -1 and cur_node.tot_weight <= capacity :
            if -cur_node.fun > best_profit:
                best_profit = -cur_node.fun
                #print("!!!!!", best_profit)
                best_backpack = cur_node.items
                
        elif -cur_node.fun < best_profit or cur_node.num_ind == -1 :
            continue
            
        else:
            
            A_equal = copy.deepcopy(cur_node.A_equal)
            b_equal_1 = copy.deepcopy(cur_node.b_equal)
            b_equal_0 = copy.deepcopy(cur_node.b_equal)

            equal = []

            for i in range(len(weights)):
                if i == cur_node.num_ind:
                    equal.append(1)
                else:
                    equal.append(0)
                    
            A_equal.append(equal)
            b_equal_1.append(1)
            b_equal_0.append(0)
            
            new_node_1=node(func, weights, profits, bounds, capacity, A_equal, b_equal_1)
            queue.append(new_node_1)
            new_node_0=node(func, weights, profits, bounds, capacity, A_equal, b_equal_0)
            queue.append(new_node_0)
            

    total_profit = 0
    best_backpack_out=[]
    for i in range(len(best_backpack)):
        if best_backpack[i] == 1:
            total_profit+= profits[i]
            best_backpack_out.append(i)

            
    return(best_backpack_out, total_profit, time.time()-start, num_op)

# Сравнение

In [ ]:
class benchmark:
  knapsnack_algorithms = [dynamic_programming_algo, ptas, approx_2, bbm]
  weights = {}
  profits = {}
  answers = {}
  capacities = {}
  numbers = range(1, 8)

  def __init__(self, path): # path to a folder, which contains benchmark files
    print("Opening up input files...")
    for filename in sorted(os.listdir(path)):
      input = open(path + '/' + filename).read()
      if "_c" in filename:
        self.capacities[filename[2:3]] = int(input)
      elif "_p" in filename:
        self.profits[filename[2:3]] = list(map(int, input.split()))
      elif "_s" in filename:
        answers_onehot = list(map(int, input.split()))
        self.answers[filename[2:3]] = []
        for i in range(len(answers_onehot)):    
          if answers_onehot[i] == 1:            
            self.answers[filename[2:3]].append(i)

        #self.answers[filename[2:3]] = list(map(int, input.split()))
      elif "_w" in filename:
        self.weights[filename[2:3]] = list(map(int, input.split()))
    print("Finished")      


  def check_single_benchmark(self, capacity, weights, profits, knapsnack_algorithm, times_repeat=10): 
    mean_time = 0
    
    for i in range(times_repeat):
      if knapsnack_algorithm.__name__ == "ptas":
        items, resulting_profit, time, num_operations = knapsnack_algorithm(capacity, weights, profits, 2)
      else:
        items, resulting_profit, time, num_operations = knapsnack_algorithm(capacity, weights, profits)
      mean_time += time

    mean_time = mean_time/times_repeat

    return items, resulting_profit, mean_time, num_operations


  def calculate_weight(self, items, weights):
    weight = 0
    for item in items:
      weight += weights[item]
    return weight

  def report(self):

    resulting_profit_table = []
    items_table = []
    times_table = []
    operations_table = []
    weights_table = []

    for algorithm in self.knapsnack_algorithms:
      print("Checking", algorithm.__name__)

      
      items_row = {}
      resulting_profit_row = {}
      times_row = {}
      operations_row = {}
      weights_row = {}

      for num in self.numbers:
        results = self.check_single_benchmark(self.capacities[str(num)], self.weights[str(num)], self.profits[str(num)], algorithm)
        
        items_row[str(num)] = results[0]
        resulting_profit_row[str(num)] = results[1]
        times_row[str(num)] = results[2]          
        operations_row[str(num)]  = results[3]
        weights_row[str(num)] = self.calculate_weight(results[0], self.weights[str(num)])

      resulting_profit_table.append(resulting_profit_row)
      items_table.append(items_row)
      times_table.append(times_row) 
      operations_table.append(operations_row)
      weights_table.append(weights_row)

    items_table.append(self.answers) # row with optimal selection of weights
              
    resulting_profit_table=pd.DataFrame(resulting_profit_table)
    items_table=pd.DataFrame(items_table)
    times_table=pd.DataFrame(times_table)
    operations_table=pd.DataFrame(operations_table)
    weights_table=pd.DataFrame(weights_table)
    for i in range(len(self.knapsnack_algorithms)):
      resulting_profit_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      items_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      times_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      operations_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      weights_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
    items_table.rename(index={len(self.knapsnack_algorithms):"Optimal selection"}, inplace=True) # row with optimal selection of weights

    

    return resulting_profit_table, items_table, times_table, operations_table, weights_table

In [ ]:
bench = benchmark('/drive/MyDrive/knapsnack_benchmarks')
resulting_profit_table, items_table, times_table, operations_table, weights_table = bench.report()


Opening up input files...
Finished
Checking dynamic_programming_algo
Checking ptas
Checking approx_2
Checking bbm


## Оптимальная ценность:

In [ ]:
resulting_profit_table

,1,2,3,4,5,6,7
dynamic_programming_algo,309,51,150,107,900,1735,1458
ptas,309,51,150,107,900,1652,1361
approx_2,309,47,146,107,888,1735,1441
bbm,309,51,150,107,900,1735,1458


## Содержимое оптимального рюкзака:

In [ ]:
items_table

,1,2,3,4,5,6,7
dynamic_programming_algo,"[0, 1, 2, 3, 5]","[1, 2, 3]","[0, 1, 4]","[0, 3]","[0, 2, 3, 4, 6, 7]","[1, 3, 6]","[0, 2, 4, 6, 7, 8, 13, 14]"
ptas,"[0, 1, 2, 3, 5]","[1, 2, 3]","[0, 1, 4]","[0, 3]","[0, 2, 3, 4, 6, 7]","[0, 3, 6]","[0, 1, 2, 3, 4, 5, 13, 14]"
approx_2,"[0, 1, 2, 3, 5]","[0, 2]","[0, 1, 3]","[0, 3]","[1, 2, 3, 5, 6, 7]","[1, 3, 6]","[0, 1, 2, 6, 7, 8, 13, 14]"
bbm,"[0, 1, 2, 3, 5]","[1, 2, 3]","[0, 1, 4]","[0, 3]","[0, 2, 3, 4, 6, 7]","[1, 3, 6]","[0, 2, 4, 6, 7, 8, 13, 14]"
Optimal selection,"[0, 1, 2, 3, 5]","[1, 2, 3]","[0, 1, 4]","[0, 3]","[0, 2, 3, 4, 6, 7]","[1, 3, 6]","[0, 2, 4, 6, 7, 8, 13, 14]"


## Время выполнения:

In [ ]:
times_table

,1,2,3,4,5,6,7
dynamic_programming_algo,0.001335,0.000094,0.001033,0.000238,0.000658,0.000863,0.010723
ptas,0.000114,0.000021,0.000032,0.000050,0.000086,0.000048,0.000561
approx_2,0.000076,0.000050,0.000133,0.000024,0.000029,0.000029,0.000035
bbm,0.352461,0.048662,0.060610,0.120725,0.087674,0.288385,7.364806


## Количество операций:

In [ ]:
operations_table

,1,2,3,4,5,6,7
dynamic_programming_algo,2949,250,2132,672,1643,2194,21848
ptas,87,20,30,41,56,42,210
approx_2,20,10,12,14,16,14,30
bbm,61,17,17,31,21,129,975


## Вес рюкзака:

In [ ]:
weights_table

,1,2,3,4,5,6,7
dynamic_programming_algo,165,26,190,50,104,169,749
ptas,165,26,190,50,104,160,707
approx_2,165,23,179,50,92,169,740
bbm,165,26,190,50,104,169,749
